# DSE511. Project 3. Part3. Modeling. 
## Code for Gaussian Processes modeling.
### Albina Jetybayeva

Gaussian Processes (GP) are a generic supervised learning method designed to solve regression and probabilistic classification problems. Gaussian processing (GP) is quite a useful technique that enables a non-parametric Bayesian approach to modeling. It has wide applicability in areas such as regression, classification, optimization, etc.

The advantages of Gaussian processes are:

- The prediction interpolates the observations (at least for regular kernels).
- The prediction is probabilistic (Gaussian) so that one can compute empirical confidence intervals and decide based on those if one should refit (online fitting, adaptive fitting) the prediction in some region of interest.
- Versatile: different kernels can be specified. Common kernels are provided, but it is also possible to specify custom kernels.

The disadvantages of Gaussian processes include:

- They are not sparse, i.e., they use the whole samples/features information to perform the prediction.
- They lose efficiency in high dimensional spaces – namely when the number of features exceeds a few dozens.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

In [2]:
# Getting the raw data
df = pd.read_csv('housing.csv') # Notice: Raw data is in the Data folder
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


## Data preprocessing

In [3]:
# Remove capped values of prices USD 500000
df=df[df['median_house_value'] < 490000]
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


In [4]:
print('There are {} rows and {} columns in train'.format(df.shape[0],df.shape[1]))

There are 19608 rows and 10 columns in train


In [5]:
# As it was dsicussed in Part 1. Explanatory Data Analysis, it might be interesting to add the possibly helpful 
#attributes combinations and study their effect on modeling too
df["rooms_per_household"] = df["total_rooms"]/df["households"]
df["bedrooms_per_room"] = df["total_bedrooms"]/df["total_rooms"]
df["population_per_household"]=df["population"]/df["households"]
df

<ipython-input-5-2953a84ef91e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["rooms_per_household"] = df["total_rooms"]/df["households"]
<ipython-input-5-2953a84ef91e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["bedrooms_per_room"] = df["total_bedrooms"]/df["total_rooms"]
<ipython-input-5-2953a84ef91e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,6.984127,0.146591,2.555556
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,6.238137,0.155797,2.109842
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,8.288136,0.129516,2.802260
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,5.817352,0.184458,2.547945
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,6.281853,0.172096,2.181467
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND,5.045455,0.224625,2.560606
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND,6.114035,0.215208,3.122807
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND,5.205543,0.215173,2.325635
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND,5.329513,0.219892,2.123209


In [6]:
# Asssigning numerical values to ocean proximity in the gradient order: the lower the number the further away is the house from the ocean
# this gradient is chosen for the better and easier interpretation of models results (feature importances)

df.loc[df['ocean_proximity'] == 'NEAR OCEAN', 'ocean_proximity'] = 4
df.loc[df['ocean_proximity'] == 'NEAR BAY', 'ocean_proximity'] = 3
df.loc[df['ocean_proximity'] == '<1H OCEAN', 'ocean_proximity'] = 2
df.loc[df['ocean_proximity'] == 'INLAND', 'ocean_proximity'] = 1
df.loc[df['ocean_proximity'] == 'ISLAND', 'ocean_proximity'] = 0

df

C:\Users\PC\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,3,6.984127,0.146591,2.555556
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,3,6.238137,0.155797,2.109842
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,3,8.288136,0.129516,2.802260
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,3,5.817352,0.184458,2.547945
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,3,6.281853,0.172096,2.181467
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,1,5.045455,0.224625,2.560606
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,1,6.114035,0.215208,3.122807
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,1,5.205543,0.215173,2.325635
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,1,5.329513,0.219892,2.123209


In [7]:
# Splitting the data into training and testing sets.
train_set1, test_set1 = train_test_split(df, test_size=0.2, random_state=1)
print("Training Data", len(train_set1))
print("Testing Data", len(test_set1))

Training Data 15686
Testing Data 3922


In [8]:
# Observing missing values train set
missing_values_count = train_set1.isnull().sum()
missing_values_count[:]

total_cells   = np.product(train_set1.shape)
total_missing = missing_values_count.sum()
percent_missing = (total_missing/total_cells)*100
print('Percent of data that is missing:', percent_missing)

imputer = SimpleImputer(strategy = "median")
housing_numerical_attributes = train_set1.drop("ocean_proximity", axis = 1) # We need to see how to put this back.
imputer.fit(housing_numerical_attributes)  
X = imputer.transform(housing_numerical_attributes)

Percent of data that is missing: 0.1431948136015457


In [9]:
# Observing missing values test set
missing_values_count = test_set1.isnull().sum()
missing_values_count[:]

total_cells   = np.product(test_set1.shape)
total_missing = missing_values_count.sum()
percent_missing = (total_missing/total_cells)*100
print('Percent of data that is missing:', percent_missing)

imputer = SimpleImputer(strategy = "median")
housing_numerical_attributes1 = test_set1.drop("ocean_proximity", axis = 1) # We need to see how to put this back.
imputer.fit(housing_numerical_attributes)  
X1 = imputer.transform(housing_numerical_attributes1)

Percent of data that is missing: 0.21182285333228731


In [10]:
# Data with replaced NA values.
# I changed the name from new_df (in original code) to housing.
train_set = pd.DataFrame(X, columns = housing_numerical_attributes.columns, index = housing_numerical_attributes.index)

train_set.insert(9,"ocean_proximity",df["ocean_proximity"],True)
train_set

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household
2736,-115.57,32.79,50.0,1291.0,277.0,864.0,274.0,1.6667,68100.0,1,4.711679,0.214562,3.153285
17659,-121.88,37.28,33.0,2951.0,529.0,1288.0,521.0,4.1554,313100.0,2,5.664107,0.179261,2.472169
17654,-121.89,37.25,21.0,2080.0,352.0,1040.0,325.0,5.2887,264500.0,2,6.400000,0.169231,3.200000
17544,-121.87,37.34,52.0,1170.0,215.0,604.0,207.0,2.6667,325900.0,2,5.652174,0.183761,2.917874
8772,-118.34,33.80,33.0,2194.0,469.0,987.0,397.0,5.0951,318900.0,2,5.526448,0.213765,2.486146
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11575,-118.00,33.77,28.0,2401.0,503.0,1155.0,456.0,3.5139,211700.0,2,5.265351,0.209496,2.532895
18228,-122.09,37.41,18.0,1476.0,473.0,838.0,415.0,3.5750,274000.0,3,3.556627,0.320461,2.019277
5425,-118.42,34.02,34.0,2243.0,444.0,973.0,413.0,4.9676,414100.0,2,5.430993,0.197949,2.355932
12799,-121.45,38.61,32.0,2436.0,612.0,1509.0,618.0,1.0424,81400.0,1,3.941748,0.251232,2.441748


In [11]:
# Data with replaced NA values.
# I changed the name from new_df (in original code) to housing.
test_set = pd.DataFrame(X1, columns = housing_numerical_attributes1.columns, index = housing_numerical_attributes1.index)

test_set.insert(9,"ocean_proximity",df["ocean_proximity"],True)
test_set

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household
11212,-117.91,33.82,29.0,1444.0,326.0,1038.0,271.0,2.3843,182900.0,2,5.328413,0.225762,3.830258
10317,-117.82,33.84,25.0,1788.0,203.0,676.0,217.0,10.1299,454300.0,2,8.239631,0.113535,3.115207
5086,-118.28,33.98,47.0,865.0,193.0,782.0,217.0,2.2411,93000.0,2,3.986175,0.223121,3.603687
7988,-118.18,33.85,38.0,3596.0,862.0,2416.0,832.0,3.6897,169800.0,2,4.322115,0.239711,2.903846
14480,-117.25,32.82,19.0,5255.0,762.0,1773.0,725.0,7.8013,474000.0,4,7.248276,0.145005,2.445517
...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,-122.29,37.90,49.0,1283.0,238.0,576.0,236.0,3.3333,276800.0,3,5.436441,0.185503,2.440678
11924,-117.39,33.95,35.0,1599.0,284.0,721.0,287.0,4.1250,120700.0,1,5.571429,0.177611,2.512195
13047,-121.28,38.55,35.0,7088.0,1279.0,4885.0,1272.0,2.6981,112500.0,1,5.572327,0.180446,3.840409
10331,-117.77,33.84,5.0,4380.0,715.0,1913.0,741.0,6.7274,266400.0,2,5.910931,0.163242,2.581646


In [12]:
train_set_without_target = train_set.drop("median_house_value", axis=1) # drop labels for training set
train_set_without_target

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household
2736,-115.57,32.79,50.0,1291.0,277.0,864.0,274.0,1.6667,1,4.711679,0.214562,3.153285
17659,-121.88,37.28,33.0,2951.0,529.0,1288.0,521.0,4.1554,2,5.664107,0.179261,2.472169
17654,-121.89,37.25,21.0,2080.0,352.0,1040.0,325.0,5.2887,2,6.400000,0.169231,3.200000
17544,-121.87,37.34,52.0,1170.0,215.0,604.0,207.0,2.6667,2,5.652174,0.183761,2.917874
8772,-118.34,33.80,33.0,2194.0,469.0,987.0,397.0,5.0951,2,5.526448,0.213765,2.486146
...,...,...,...,...,...,...,...,...,...,...,...,...
11575,-118.00,33.77,28.0,2401.0,503.0,1155.0,456.0,3.5139,2,5.265351,0.209496,2.532895
18228,-122.09,37.41,18.0,1476.0,473.0,838.0,415.0,3.5750,3,3.556627,0.320461,2.019277
5425,-118.42,34.02,34.0,2243.0,444.0,973.0,413.0,4.9676,2,5.430993,0.197949,2.355932
12799,-121.45,38.61,32.0,2436.0,612.0,1509.0,618.0,1.0424,1,3.941748,0.251232,2.441748


In [13]:
test_set_without_target = test_set.drop("median_house_value", axis=1) # drop labels for training set
test_set_without_target

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household
11212,-117.91,33.82,29.0,1444.0,326.0,1038.0,271.0,2.3843,2,5.328413,0.225762,3.830258
10317,-117.82,33.84,25.0,1788.0,203.0,676.0,217.0,10.1299,2,8.239631,0.113535,3.115207
5086,-118.28,33.98,47.0,865.0,193.0,782.0,217.0,2.2411,2,3.986175,0.223121,3.603687
7988,-118.18,33.85,38.0,3596.0,862.0,2416.0,832.0,3.6897,2,4.322115,0.239711,2.903846
14480,-117.25,32.82,19.0,5255.0,762.0,1773.0,725.0,7.8013,4,7.248276,0.145005,2.445517
...,...,...,...,...,...,...,...,...,...,...,...,...
385,-122.29,37.90,49.0,1283.0,238.0,576.0,236.0,3.3333,3,5.436441,0.185503,2.440678
11924,-117.39,33.95,35.0,1599.0,284.0,721.0,287.0,4.1250,1,5.571429,0.177611,2.512195
13047,-121.28,38.55,35.0,7088.0,1279.0,4885.0,1272.0,2.6981,1,5.572327,0.180446,3.840409
10331,-117.77,33.84,5.0,4380.0,715.0,1913.0,741.0,6.7274,2,5.910931,0.163242,2.581646


In [14]:
# Creating pandas series full of zeros to store the standard deviation and the mean from the training set.
std_dev_tr= pd.Series({col:0 for col in train_set_without_target.columns}, dtype="float32")
mean_tr= pd.Series({col:0 for col in train_set_without_target.columns}, dtype="float32")

# Getting the values for the mean and standard deviation from the training dataset.
for col in train_set_without_target.columns:
    std_dev_tr[col]= train_set_without_target[col].std()
    mean_tr[col]= train_set_without_target[col].mean()
    # Changing the training data so it is normalized with the mean and standard deviation from the training set.
    train_set_without_target[col]=(train_set_without_target[col]-mean_tr[col])/std_dev_tr[col]

for col in test_set_without_target.columns:
    # Changing the testing data so it is normalized with the mean and standard deviation from the training set.
    test_set_without_target[col]=(test_set_without_target[col]-mean_tr[col])/std_dev_tr[col]

In [15]:
train_set_without_target

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household
2736,1.987142,-1.326644,1.738389,-0.614893,-0.625995,-0.516708,-0.595592,-1.281953,-1.054676,-0.299836,-0.005576,0.003533
17659,-1.162709,0.760676,0.376473,0.155700,-0.021575,-0.135922,0.053373,0.314878,-0.021924,0.141273,-0.627446,-0.054483
17654,-1.167700,0.746729,-0.584880,-0.248629,-0.446108,-0.358646,-0.461596,1.042040,-0.021924,0.482095,-0.804145,0.007512
17544,-1.157717,0.788569,1.898615,-0.671063,-0.774701,-0.750208,-0.771628,-0.640321,-0.021924,0.135746,-0.548183,-0.016519
8772,0.604403,-0.857113,0.376473,-0.195709,-0.165485,-0.406244,-0.272424,0.917820,-0.021924,0.077517,-0.019626,-0.053292
...,...,...,...,...,...,...,...,...,...,...,...,...
11575,0.774125,-0.871060,-0.024091,-0.099617,-0.083936,-0.255367,-0.117408,-0.096730,-0.021924,-0.043408,-0.094825,-0.049310
18228,-1.267537,0.821111,-0.825218,-0.529014,-0.155891,-0.540058,-0.225131,-0.057526,1.010828,-0.834789,1.859945,-0.093059
5425,0.564468,-0.754839,0.456585,-0.172963,-0.225447,-0.418817,-0.230385,0.836012,-0.021924,0.033308,-0.298237,-0.064384
12799,-0.948059,1.378969,0.296360,-0.083370,0.177499,0.062553,0.308229,-1.682525,-1.054676,-0.656423,0.640393,-0.057074


In [16]:
train_set_without_target.insert(12,"median_house_value",train_set["median_house_value"])
train_set_without_target

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household,median_house_value
2736,1.987142,-1.326644,1.738389,-0.614893,-0.625995,-0.516708,-0.595592,-1.281953,-1.054676,-0.299836,-0.005576,0.003533,68100.0
17659,-1.162709,0.760676,0.376473,0.155700,-0.021575,-0.135922,0.053373,0.314878,-0.021924,0.141273,-0.627446,-0.054483,313100.0
17654,-1.167700,0.746729,-0.584880,-0.248629,-0.446108,-0.358646,-0.461596,1.042040,-0.021924,0.482095,-0.804145,0.007512,264500.0
17544,-1.157717,0.788569,1.898615,-0.671063,-0.774701,-0.750208,-0.771628,-0.640321,-0.021924,0.135746,-0.548183,-0.016519,325900.0
8772,0.604403,-0.857113,0.376473,-0.195709,-0.165485,-0.406244,-0.272424,0.917820,-0.021924,0.077517,-0.019626,-0.053292,318900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11575,0.774125,-0.871060,-0.024091,-0.099617,-0.083936,-0.255367,-0.117408,-0.096730,-0.021924,-0.043408,-0.094825,-0.049310,211700.0
18228,-1.267537,0.821111,-0.825218,-0.529014,-0.155891,-0.540058,-0.225131,-0.057526,1.010828,-0.834789,1.859945,-0.093059,274000.0
5425,0.564468,-0.754839,0.456585,-0.172963,-0.225447,-0.418817,-0.230385,0.836012,-0.021924,0.033308,-0.298237,-0.064384,414100.0
12799,-0.948059,1.378969,0.296360,-0.083370,0.177499,0.062553,0.308229,-1.682525,-1.054676,-0.656423,0.640393,-0.057074,81400.0


In [17]:
train=train_set_without_target
train

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household,median_house_value
2736,1.987142,-1.326644,1.738389,-0.614893,-0.625995,-0.516708,-0.595592,-1.281953,-1.054676,-0.299836,-0.005576,0.003533,68100.0
17659,-1.162709,0.760676,0.376473,0.155700,-0.021575,-0.135922,0.053373,0.314878,-0.021924,0.141273,-0.627446,-0.054483,313100.0
17654,-1.167700,0.746729,-0.584880,-0.248629,-0.446108,-0.358646,-0.461596,1.042040,-0.021924,0.482095,-0.804145,0.007512,264500.0
17544,-1.157717,0.788569,1.898615,-0.671063,-0.774701,-0.750208,-0.771628,-0.640321,-0.021924,0.135746,-0.548183,-0.016519,325900.0
8772,0.604403,-0.857113,0.376473,-0.195709,-0.165485,-0.406244,-0.272424,0.917820,-0.021924,0.077517,-0.019626,-0.053292,318900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11575,0.774125,-0.871060,-0.024091,-0.099617,-0.083936,-0.255367,-0.117408,-0.096730,-0.021924,-0.043408,-0.094825,-0.049310,211700.0
18228,-1.267537,0.821111,-0.825218,-0.529014,-0.155891,-0.540058,-0.225131,-0.057526,1.010828,-0.834789,1.859945,-0.093059,274000.0
5425,0.564468,-0.754839,0.456585,-0.172963,-0.225447,-0.418817,-0.230385,0.836012,-0.021924,0.033308,-0.298237,-0.064384,414100.0
12799,-0.948059,1.378969,0.296360,-0.083370,0.177499,0.062553,0.308229,-1.682525,-1.054676,-0.656423,0.640393,-0.057074,81400.0


In [18]:
test_set_without_target.insert(12,"median_house_value",test_set["median_house_value"])
test_set_without_target

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household,median_house_value
11212,0.819052,-0.847816,0.056022,-0.543869,-0.508469,-0.360442,-0.603475,-0.821518,-0.021924,-0.014201,0.191714,0.061195,182900.0
10317,0.863978,-0.838518,-0.264429,-0.384179,-0.803483,-0.685546,-0.745354,4.148312,-0.021924,1.334104,-1.785296,0.000289,454300.0
5086,0.634354,-0.773435,1.498051,-0.812648,-0.827468,-0.590350,-0.745354,-0.913400,-0.021924,-0.635847,0.145201,0.041896,93000.0
7988,0.684272,-0.833869,0.777036,0.455117,0.777122,0.877110,0.870491,0.016069,-0.021924,-0.480259,0.437442,-0.017714,169800.0
14480,1.148513,-1.312698,-0.745106,1.225246,0.537273,0.299646,0.589360,2.654206,2.04358,0.874967,-1.230914,-0.056753,474000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,-1.367374,1.048903,1.658276,-0.618607,-0.719536,-0.775354,-0.695433,-0.212608,1.010828,0.035831,-0.517495,-0.057165,276800.0
11924,1.078627,-0.787381,0.536698,-0.471916,-0.609205,-0.645133,-0.561436,0.295372,-1.054676,0.098349,-0.656517,-0.051074,120700.0
13047,-0.863198,1.351076,0.536698,2.076147,1.777292,3.094466,2.026542,-0.620174,-1.054676,0.098766,-0.606578,0.062060,112500.0
10331,0.888937,-0.838518,-1.866684,0.819060,0.424544,0.425377,0.631398,1.965157,-0.021924,0.255587,-0.909643,-0.045158,266400.0


In [19]:
test=test_set_without_target
test

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household,median_house_value
11212,0.819052,-0.847816,0.056022,-0.543869,-0.508469,-0.360442,-0.603475,-0.821518,-0.021924,-0.014201,0.191714,0.061195,182900.0
10317,0.863978,-0.838518,-0.264429,-0.384179,-0.803483,-0.685546,-0.745354,4.148312,-0.021924,1.334104,-1.785296,0.000289,454300.0
5086,0.634354,-0.773435,1.498051,-0.812648,-0.827468,-0.590350,-0.745354,-0.913400,-0.021924,-0.635847,0.145201,0.041896,93000.0
7988,0.684272,-0.833869,0.777036,0.455117,0.777122,0.877110,0.870491,0.016069,-0.021924,-0.480259,0.437442,-0.017714,169800.0
14480,1.148513,-1.312698,-0.745106,1.225246,0.537273,0.299646,0.589360,2.654206,2.04358,0.874967,-1.230914,-0.056753,474000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,-1.367374,1.048903,1.658276,-0.618607,-0.719536,-0.775354,-0.695433,-0.212608,1.010828,0.035831,-0.517495,-0.057165,276800.0
11924,1.078627,-0.787381,0.536698,-0.471916,-0.609205,-0.645133,-0.561436,0.295372,-1.054676,0.098349,-0.656517,-0.051074,120700.0
13047,-0.863198,1.351076,0.536698,2.076147,1.777292,3.094466,2.026542,-0.620174,-1.054676,0.098766,-0.606578,0.062060,112500.0
10331,0.888937,-0.838518,-1.866684,0.819060,0.424544,0.425377,0.631398,1.965157,-0.021924,0.255587,-0.909643,-0.045158,266400.0


In [20]:
housing = train.drop("median_house_value", axis=1) # drop labels for training set
housing_labels = train["median_house_value"].copy()

housing_t = test.drop("median_house_value", axis=1) # drop labels for test set
housing_labels_t = test["median_house_value"].copy()

## Modeling with GP

In [21]:
#Here we will try a radial-basis function kernel with noise and an offset. 
#The hyperparameters for the kernel are suggested values and these will be optimized during fitting.
from sklearn.metrics import r2_score
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, WhiteKernel

kernel = ConstantKernel(1.0) + ConstantKernel(1.0) * RBF(10)  + WhiteKernel(5)
model = GaussianProcessRegressor(kernel=kernel)
model.fit(housing, housing_labels)
y_pred_tr, y_pred_tr_std = model.predict(housing, return_std=True)
y_pred_te, y_pred_te_std = model.predict(housing_t, return_std=True)

KeyboardInterrupt: 

### As the dataset is too large, the model cannot run on this computer and the code crashes. The warning that is receieved: Unable to allocate 3.98 GiB for an array with shape (15686, 15686, 2) and data type float64.

This happens due to disadvantages of Gaussian processes mentioned before: GPs are not sparse, i.e., they use the whole samples/features information to perform the prediction; They lose efficiency in high dimensional spaces – namely when the number of features exceeds a few dozens.

To check this models performance on the smaller dataset, our dataset will be cut to 800 train and 200 test sets and used for GP again.

In [22]:
h=housing.iloc[:800]
h

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household
2736,1.987142,-1.326644,1.738389,-0.614893,-0.625995,-0.516708,-0.595592,-1.281953,-1.054676,-0.299836,-0.005576,0.003533
17659,-1.162709,0.760676,0.376473,0.155700,-0.021575,-0.135922,0.053373,0.314878,-0.021924,0.141273,-0.627446,-0.054483
17654,-1.167700,0.746729,-0.584880,-0.248629,-0.446108,-0.358646,-0.461596,1.042040,-0.021924,0.482095,-0.804145,0.007512
17544,-1.157717,0.788569,1.898615,-0.671063,-0.774701,-0.750208,-0.771628,-0.640321,-0.021924,0.135746,-0.548183,-0.016519
8772,0.604403,-0.857113,0.376473,-0.195709,-0.165485,-0.406244,-0.272424,0.917820,-0.021924,0.077517,-0.019626,-0.053292
...,...,...,...,...,...,...,...,...,...,...,...,...
3294,-1.547080,1.555624,-0.985444,0.772174,0.990587,0.140686,0.434344,-1.159787,-1.054676,0.493639,0.129818,-0.060937
15071,1.283293,-1.326644,0.296360,0.529391,0.297424,0.154157,0.255682,0.129638,-0.021924,0.426952,-0.680469,-0.035590
17611,-1.182676,0.765325,0.536698,-0.198030,-0.554040,-0.574185,-0.474733,2.567329,-0.021924,0.686167,-1.314733,-0.052112
15060,1.308252,-1.326644,-0.745106,0.342777,-0.244635,0.456809,0.476382,-0.414145,-0.021924,-0.204332,-0.183681,-0.021763


In [23]:
h_l=housing_labels.iloc[:800]
h_l

2736      68100.0
17659    313100.0
17654    264500.0
17544    325900.0
8772     318900.0
           ...   
3294      75900.0
15071    189700.0
17611    426900.0
15060    142300.0
2120      67900.0
Name: median_house_value, Length: 800, dtype: float64

In [24]:
h_t=housing_t.iloc[:200]
h_t

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household
11212,0.819052,-0.847816,0.056022,-0.543869,-0.508469,-0.360442,-0.603475,-0.821518,-0.021924,-0.014201,0.191714,0.061195
10317,0.863978,-0.838518,-0.264429,-0.384179,-0.803483,-0.685546,-0.745354,4.148312,-0.021924,1.334104,-1.785296,0.000289
5086,0.634354,-0.773435,1.498051,-0.812648,-0.827468,-0.590350,-0.745354,-0.913400,-0.021924,-0.635847,0.145201,0.041896
7988,0.684272,-0.833869,0.777036,0.455117,0.777122,0.877110,0.870491,0.016069,-0.021924,-0.480259,0.437442,-0.017714
14480,1.148513,-1.312698,-0.745106,1.225246,0.537273,0.299646,0.589360,2.654206,2.04358,0.874967,-1.230914,-0.056753
...,...,...,...,...,...,...,...,...,...,...,...,...
8384,0.594419,-0.787381,-0.184317,0.285679,1.321579,1.574917,1.364440,-0.648790,-0.021924,-1.014941,2.152138,0.001583
2558,-2.315823,2.383114,0.136134,0.166841,0.230266,-0.064974,0.216271,-0.784496,2.04358,-0.118638,-0.031175,-0.065335
7146,0.704239,-0.754839,1.337825,-0.607466,-0.611604,-0.424205,-0.648140,-0.586873,-0.021924,-0.098837,0.029023,0.059221
11222,0.824043,-0.857113,-0.104204,-0.204065,-0.230244,0.000586,-0.217248,0.455780,-0.021924,-0.071015,-0.207059,0.028378


In [25]:
h_t_l=housing_labels_t.iloc[:200]
h_t_l

11212    182900.0
10317    454300.0
5086      93000.0
7988     169800.0
14480    474000.0
           ...   
8384     177200.0
2558      69000.0
7146     178300.0
11222    212500.0
13845    103100.0
Name: median_house_value, Length: 200, dtype: float64

In [26]:
kernel = ConstantKernel(1.0) + ConstantKernel(1.0) * RBF(10)  + WhiteKernel(5)
model = GaussianProcessRegressor(kernel=kernel)
model.fit(h, h_l)
y_pred_tr, y_pred_tr_std = model.predict(h, return_std=True)
y_pred_te, y_pred_te_std = model.predict(h_t, return_std=True)

C:\Users\PC\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
C:\Users\PC\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
C:\Users\PC\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The opt

In [27]:
from sklearn.metrics import mean_squared_error
import numpy as np

lin_mse_en = mean_squared_error(h_t_l, y_pred_te)
lin_rmse_en = np.sqrt(lin_mse_en)
lin_rmse_en #l1_ratio=0.5

94134.30762012792

In [28]:
from sklearn.metrics import mean_absolute_error

lin_mae = mean_absolute_error(h_t_l, y_pred_te)
lin_mae

76483.56334164581

As it can be seen, this model performs worse than the baseline Linear Regression. This can be explained by the fact that not all data is used and only a small portion of it (5%), which does not allow the model to train and predict well.

The further investigation can be done on kernels and their parameters. The used ones were similar to the example, where this GP model was applied to a small Boston housing dataset from scikit-learn (https://h1ros.github.io/posts/loading-scikit-learns-boston-housing-dataset/).

In [29]:
# Instantiate a Gaussian Process model
##kernel1 = ConstantKernel(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2))
##gp = GaussianProcessRegressor(kernel=kernel1, n_restarts_optimizer=9)

# Fit to data using Maximum Likelihood Estimation of the parameters
##gp.fit(h, h_l)

# Make the prediction on the meshed x-axis (ask for MSE as well)
##y_pred, sigma = gp.predict(h_t, return_std=True)

In [30]:
#plt.figure()
#plt.errorbar(h_t_l, y_pred, yerr=sigma, fmt='o')
#plt.title('Gaussian process regression, R2=%.2f' % r2_score(h_t_l, y_pred))
#plt.xlabel('Actual')
#plt.ylabel('Predicted')

In [31]:
#lin_mse_gp = mean_squared_error(h_t_l, y_pred)
#lin_rmse_gp = np.sqrt(lin_mse_gp)
#lin_rmse_gp #l1_ratio=0.5